In [9]:
# 导入必要的包
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataloader
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


class Net(nn.Module):  # 定义了一个名为 Net 的类，它继承自 nn.Module
    def __init__(self):  # 构造函数，3x32x32的卷积层，2x2的池化层，64个全连接层
        super(Net, self).__init__()  # 调用父类的构造函数

        # 定义网络结构
        self.conv1 = nn.Conv2d(3,6,3)  # 第一个卷积层，输入通道为3，输出通道为6，卷积核大小为3x3
        self.conv2 = nn.Conv2d(6,16,3)  # 第二个卷积层，输入通道为6，输出通道为16，卷积核大小为3x3
        self.fc1 = nn.Linear(16*28*28,512)  # 第一个全连接层，输入特征数为16*28*28，输出特征数为512
        self.fc2 = nn.Linear(512,64)  # 第二个全连接层，输入特征数为512，输出特征数为64
        self.fc3 = nn.Linear(64,10)  # 第三个全连接层，输入特征数为64，输出特征数为10

    def forward(self, x):
        # 定义前向传播过程
        x=self.conv1(x)  # 卷积层 1
        x=F.relu(x)  # 使用 ReLU 激活函数

        x=self.conv2(x)  # 卷积层 2
        x=F.relu(x)  # 使用 ReLU 激活函数

        x=x.view(-1,16*28*28)  # 对输入进行展平操作
        x=self.fc1(x)  # 全连接层 1
        x=F.relu(x)  # 使用 ReLU 激活函数

        x=self.fc2(x)  # 全连接层 2
        x=F.relu(x)  # 使用 ReLU 激活函数

        x=self.fc3(x)  # 全连接层 3
        return x  # 返回输出


net = Net()  # 创建一个名为net的神经网络实例
print(net)  # 打印神经网络实例

Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=12544, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [10]:
transform = transforms.Compose([transforms.ToTensor(),  # 将 PIL 图像或 numpy.ndarray 转换为张量
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 标准化图像的张量

# 构建训练数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,  # 指定训练数据集，如果不存在则从互联网上下载
                                        download=True, transform=transform)  # 使用前面定义的转换函数了，如果是私人数据集就把cifar10以及后面改了
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,  # 创建训练数据的数据加载器，每个批次包含4个样本
                                          shuffle=True,  # 对数据进行洗牌
                                          num_workers=2)  # 使用多个子进程来加载数据，加快数据加载速度

# 构建测试数据集
testset = torchvision.datasets.CIFAR10(root='./data', train=False,  # 指定测试数据集，如果不存在则从互联网上下载
                                       download=True, transform=transform)  # 使用前面定义的转换函数
testloader = torch.utils.data.DataLoader(testset, batch_size=4,  # 创建测试数据的数据加载器，每个批次包含4个样本
                                         shuffle=False,  # 不对数据进行洗牌
                                         num_workers=2)  # 使用多个子进程来加载数据，加快数据加载速度

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()  # 定义损失函数为交叉熵
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)  # 定义优化器为随机梯度下降，学习率为0.1


In [ ]:
# mini batch训练神经网络
for epoch in range(2):
    for i, data in enumerate(trainloader):
        images, labels = data
        outputs = net(images)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        optimizer.zero_grad()  # 梯度清零
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        
        if(i%1000==0):
            print('Epoch: %d, Step: /%d, Loss: %.3f' %(epoch, i, loss.item()))

In [13]:
# 测试模型,需要知道准确率，一共有多少个正确的判断除以总共的样本数
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)     # 把图像给到神经网络，得到输出
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

print('准确率：',float(correct) / total)


准确率： 0.0998


In [ ]:
# 保存模型，包含所有参数
torch.save(net.state_dict(), 'net.pth')

In [ ]:
# 把刚才保存的模型读取到新的网络中，并测试准确率
net2.load_state_dict(torch.load('net.pth'))

In [20]:
# 加载读取模型，并使用
net2 = Net()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net2(images)     # 把新导入的图像给到神经网络，得到输出
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

print('准确率：',float(correct) / total)

准确率： 0.0707
